In [7]:
from json import loads
from pymongo import MongoClient, GEOSPHERE
from os import getenv

In [8]:
password = getenv('MONGODB_ROOT_PASSWORD')
client = MongoClient('mongodb://root:{}@my-mongodb:27017'.format(password))
db = client['geotest-db']

In [9]:
geo_collection = db['2020_pop__geojson']

Create an index on the geometry property, which will contain all geographical data. Makes searching faster.

In [10]:
geo_collection.create_index([('geometry', GEOSPHERE)])

'geometry_2dsphere'

Consume all Kafka messages one-by-one and insert values into database.

In [11]:
from kafka import KafkaConsumer
from tqdm.notebook import tqdm

consumer = KafkaConsumer(
  '2020_pop_points',
  bootstrap_servers=['my-kafka:9092'], 
#   auto_offset_reset='earliest',
  enable_auto_commit=False,
  value_deserializer= lambda x: loads(x.decode('utf-8'))
)

for message in tqdm(consumer):
    geo_collection.insert_one(message.value) # deserialized to json

KeyboardInterrupt: 

In [ ]:
features = geo_collection.find({'geometry' : {'$within': {'$box' : [[100, 100],[0,0]]}}}).limit(5)

for f in features:
    print(f)

In [ ]:
features = geo_collection.find({'geometry' : {'$nearSphere': [62.21541, 34.34125]}}).limit(5)

for f in features:
    print(f)